<a target="_blank" href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL-Tutorials/blob/master/1-Introduction/China_A_share_market_tushare.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Quantitative trading in China A stock market with FinRL

Install FinRL

In [1]:
!pip install wrds
!pip install swig
# !pip install -q condacolab
# import condacolab
# condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/AI4Finance-Foundation/FinRL.git


'apt-get' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


Install other libraries

In [2]:
# 在conda shell中执行命令，安装ta-lib
# conda install -c conda-forge ta-libz

# 安装gym库
# pip install gym

In [3]:
!pip install stockstats
!pip install tushare
!pip install shimmy
!pip install gym

In [4]:
# %cd /
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
# %cd /FinRL-Meta/

## Import Modules

In [5]:
import warnings

warnings.filterwarnings("ignore")

import pandas as pd 
from IPython import display

display.set_matplotlib_formats("svg")

from meta import config 
from meta.data_processor import DataProcessor 
from main import check_and_make_directories 
from meta.data_processors.tushare import Tushare, ReturnPlotter 
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv 
from agents.stablebaselines3_models import DRLAgent 
import os 
from typing import List 
from argparse import ArgumentParser 
from meta import config 
from meta.config_tickers import DOW_30_TICKER 
from meta.config import ( DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR, INDICATORS, TRAIN_START_DATE, TRAIN_END_DATE, TEST_START_DATE, TEST_END_DATE, TRADE_START_DATE, TRADE_END_DATE, ERL_PARAMS, RLlib_PARAMS, SAC_PARAMS, ALPACA_API_KEY, ALPACA_API_SECRET, ALPACA_API_BASE_URL, )

import pyfolio
from pyfolio import timeseries

pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


## Create Folders

In [6]:
import os

''' 
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"): 
  os.makedirs("./datasets") 
if not os.path.exists("./trained_models"): 
  os.makedirs("./trained_models") 
if not os.path.exists("./tensorboard_log"): 
  os.makedirs("./tensorboard_log") 
if not os.path.exists("./results"): 
  os.makedirs("./results") 
'''

check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

## Download data, cleaning and feature engineering

In [ ]:
ticker_list = ['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH', '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH', '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

TRAIN_START_DATE = '2015-01-01' 
TRAIN_END_DATE= '2019-08-01' 
TRADE_START_DATE = '2019-08-01' 
TRADE_END_DATE = '2019-12-31'

TIME_INTERVAL = "1d" 
kwargs = {} 
kwargs['token'] = '27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5' 
p = DataProcessor(data_source='tushare', start_date=TRAIN_START_DATE, end_date=TRADE_END_DATE, time_interval=TIME_INTERVAL, **kwargs)

tushare successfully connected


### Download and Clean

In [8]:
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:08<00:00,  1.75it/s]


Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (17960, 8)
Shape of DataFrame:  (18315, 8)


### Add technical indicator

In [9]:
p.add_technical_indicator(config.INDICATORS) 
p.fillna()

#print(f"p.dataframe: {p.dataframe}")

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)


## Split training dataset

In [10]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE) 

print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [11]:
print(f"train.tic.unique(): {train.tic.unique()}")

train.tic.unique(): ['600000.SH' '600009.SH' '600016.SH' '600028.SH' '600030.SH' '600031.SH'
 '600036.SH' '600050.SH' '600104.SH' '600196.SH' '600276.SH' '600309.SH'
 '600519.SH' '600547.SH' '600570.SH']


In [12]:
print(f"train.head(): {train.head()}")

train.head():          tic        time  index   open   high    low  close  adjusted_close  \
0  600000.SH  2015-01-08     45  15.87  15.88  15.20  15.25           15.25   
0  600009.SH  2015-01-08     46  20.18  20.18  19.73  20.00           20.00   
0  600016.SH  2015-01-08     47  10.61  10.66  10.09  10.20           10.20   
0  600028.SH  2015-01-08     48   7.09   7.41   6.83   6.85            6.85   
0  600030.SH  2015-01-08     49  36.40  36.70  34.68  35.25           35.25   

       volume      macd    boll_ub    boll_lb     rsi_30      cci_30  \
0  3306271.72 -0.032571  16.617911  15.012089   6.058641 -125.593009   
0   198117.45 -0.016008  20.663897  19.736103  12.828915  -90.842491   
0  4851684.17 -0.018247  10.957604   9.997396  11.862558  -99.887006   
0  8190902.35 -0.008227   7.342000   6.743000  27.409248   36.578171   
0  6376268.69  0.032910  36.576444  33.808556  61.517448   47.947020   

        dx_30  close_30_sma  close_60_sma  
0   23.014040       15.8150       

In [13]:
print(f"train.shape: {train.shape}")

train.shape: (16695, 17)


In [14]:
stock_dimension = len(train.tic.unique()) 
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1 

print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


## Train

In [15]:
train["date"] = train["time"]

In [16]:
train

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,date
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.815000,15.815000,2015-01-08
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.200000,20.200000,2015-01-08
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.477500,10.477500,2015-01-08
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.042500,7.042500,2015-01-08
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.192500,35.192500,2015-01-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112,600276.SH,2019-07-31,16735,66.40,66.99,65.67,66.94,66.94,146493.41,0.437578,67.507281,65.693719,53.439654,-24.866183,6.630001,66.724000,64.270000,2019-07-31
1112,600309.SH,2019-07-31,16736,44.30,45.36,43.79,43.98,43.98,187447.27,0.575926,44.570125,39.813875,56.128173,146.183540,34.221232,42.393000,40.986833,2019-07-31
1112,600519.SH,2019-07-31,16737,975.00,981.77,966.59,972.60,972.60,22526.20,3.505151,999.017250,942.600750,55.068267,-17.175704,0.986972,978.084333,936.501833,2019-07-31
1112,600547.SH,2019-07-31,16738,41.81,42.46,41.61,42.13,42.13,289851.51,0.988650,42.691443,39.307557,59.527172,94.479895,33.512298,40.929000,36.397000,2019-07-31


In [17]:
env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": True, "hundred_each_trade": True }

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [18]:
env_train, _ = e_train_gym.get_sb_env() 

print(f"print(type(env_train)): {print(type(env_train))}")

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None


### DDPG

In [19]:
agent = DRLAgent(env=env_train) 
DDPG_PARAMS = { "batch_size": 256, "buffer_size": 2000, "learning_rate": 0.0005, "action_noise": "normal", } 
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300])) 
model_ddpg = agent.get_model("ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS)

trained_ddpg = agent.train_model(model=model_ddpg, tb_log_name='ddpg', total_timesteps=200)

{'batch_size': 256, 'buffer_size': 2000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device


Logging to tensorboard_log/ddpg\ddpg_5


### A2C

In [20]:
agent = DRLAgent(env=env_train) 
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(model=model_a2c, tb_log_name='a2c', total_timesteps=1000)

{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c\a2c_4
----------------------------------------
| time/                 |              |
|    fps                | 241          |
|    iterations         | 100          |
|    time_elapsed       | 2            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -21.4        |
|    explained_variance | -5.98        |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -6.52        |
|    reward             | -0.122644305 |
|    std                | 1.01         |
|    value_loss         | 0.163        |
----------------------------------------
---------------------------------------
| time/                 |             |
|    fps                | 241         |
|    iterations         | 200         |
|    time_elapsed       | 4           |
|    total_timesteps    | 1000  

## Trade

In [21]:
trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE) 

In [22]:
trade["date"] = trade["time"]

In [23]:

env_kwargs = { "stock_dim": stock_dimension, "hmax": 1000, "initial_amount": 1000000, "buy_cost_pct": 6.87e-5, "sell_cost_pct": 1.0687e-3, "reward_scaling": 1e-4, "state_space": state_space, "action_space": stock_dimension, "tech_indicator_list": config.INDICATORS, "print_verbosity": 1, "initial_buy": False, "hundred_each_trade": True } 
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

In [24]:
df_account_value, df_actions = DRLAgent.DRL_prediction(model=trained_ddpg, environment=e_trade_gym)

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1244424.44
total_reward: 244424.44
total_cost: 1059.56
total_trades: 13
Sharpe: 2.407
hit end!


In [25]:
df_actions

,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600050.SH,600104.SH,600196.SH,600276.SH,600309.SH,600519.SH,600547.SH,600570.SH
date,,,,,,,,,,,,,,,
2019-08-01,0,0,0,200,0,0,0,0,0,0,0,900,1000,0,0
2019-08-01,1000,0,0,1000,1000,900,0,0,0,0,0,1000,-900,0,0
2019-08-02,0,0,0,200,0,-900,0,0,0,0,0,500,800,0,0
2019-08-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-08-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
df_actions.to_csv("action.csv", index=False) 
print(f"df_actions: {df_actions}")

df_actions:             600000.SH  600009.SH  600016.SH  600028.SH  600030.SH  600031.SH  \
date                                                                           
2019-08-01          0          0          0        200          0          0   
2019-08-01       1000          0          0       1000       1000        900   
2019-08-02          0          0          0        200          0       -900   
2019-08-05          0          0          0          0          0          0   
2019-08-06          0          0          0          0          0          0   
...               ...        ...        ...        ...        ...        ...   
2019-12-24          0          0          0          0          0          0   
2019-12-25          0          0          0          0          0          0   
2019-12-26          0          0          0          0          0          0   
2019-12-27          0          0          0          0          0          0   
2019-12-30          0       

## Backtest

### matplotlib inline

In [27]:
df_account_value
df_account_value["time"] = df_account_value["date"]

In [28]:
df_account_value

,date,account_value,time
0,2019-08-01,9.999313e+05,2019-08-01
1,2019-08-02,9.937678e+05,2019-08-02
2,2019-08-05,9.878834e+05,2019-08-05
3,2019-08-06,9.911284e+05,2019-08-06
4,2019-08-07,9.894164e+05,2019-08-07
...,...,...,...
98,2019-12-25,1.192310e+06,2019-12-25
99,2019-12-26,1.190390e+06,2019-12-26
100,2019-12-27,1.220072e+06,2019-12-27
101,2019-12-30,1.246944e+06,2019-12-30


In [29]:
trade

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,date
0,600000.SH,2019-08-01,16740,11.76,11.85,11.63,11.65,11.65,324717.02,0.061723,11.954291,11.217709,51.629338,30.105973,9.109642,11.632667,11.516667,2019-08-01
0,600009.SH,2019-08-01,16741,82.78,83.46,81.60,82.01,82.01,72583.66,1.456486,85.100887,78.654113,57.259597,28.697822,13.903367,81.707000,76.533333,2019-08-01
0,600016.SH,2019-08-01,16742,6.04,6.06,6.01,6.02,6.02,517116.83,-0.045649,6.088116,5.935884,43.258474,-43.433030,5.774779,6.139667,6.162500,2019-08-01
0,600028.SH,2019-08-01,16743,5.25,5.27,5.21,5.23,5.23,616872.58,-0.030634,5.362514,5.179486,42.928615,-77.147867,22.469955,5.324333,5.357333,2019-08-01
0,600030.SH,2019-08-01,16744,23.09,23.19,22.81,22.92,22.92,809710.56,0.143135,23.703768,22.720232,51.752238,-102.582689,11.905549,23.385667,21.918333,2019-08-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,600276.SH,2020-01-02,18295,88.00,88.10,86.85,87.68,87.68,210809.91,-0.345742,87.726881,82.918119,53.727892,55.084385,2.193577,86.140333,87.454167,2020-01-02
103,600309.SH,2020-01-02,18296,56.50,57.60,55.69,55.89,55.89,213379.65,1.824002,56.464963,49.288037,67.318349,154.362552,50.898752,51.418667,48.306667,2020-01-02
103,600519.SH,2020-01-02,18297,1128.00,1145.06,1116.00,1130.00,1130.00,148099.16,-3.944016,1188.194602,1121.743398,46.814398,-98.839848,24.496850,1158.660667,1175.338000,2020-01-02
103,600547.SH,2020-01-02,18298,32.31,32.96,32.26,32.90,32.90,489106.37,0.348359,32.722690,30.612310,52.089075,155.084385,32.165604,31.288667,31.498333,2020-01-02


In [36]:
trade.time.unique()

array(['2019-08-01', '2019-08-02', '2019-08-05', '2019-08-06',
       '2019-08-07', '2019-08-08', '2019-08-09', '2019-08-12',
       '2019-08-13', '2019-08-14', '2019-08-15', '2019-08-16',
       '2019-08-19', '2019-08-20', '2019-08-21', '2019-08-22',
       '2019-08-23', '2019-08-26', '2019-08-27', '2019-08-28',
       '2019-08-29', '2019-08-30', '2019-09-02', '2019-09-03',
       '2019-09-04', '2019-09-05', '2019-09-06', '2019-09-09',
       '2019-09-10', '2019-09-11', '2019-09-12', '2019-09-16',
       '2019-09-17', '2019-09-18', '2019-09-19', '2019-09-20',
       '2019-09-23', '2019-09-24', '2019-09-25', '2019-09-26',
       '2019-09-27', '2019-09-30', '2019-10-08', '2019-10-09',
       '2019-10-10', '2019-10-11', '2019-10-14', '2019-10-15',
       '2019-10-16', '2019-10-17', '2019-10-18', '2019-10-21',
       '2019-10-22', '2019-10-23', '2019-10-24', '2019-10-25',
       '2019-10-28', '2019-10-29', '2019-10-30', '2019-10-31',
       '2019-11-01', '2019-11-04', '2019-11-05', '2019-

In [37]:
 df_account_value.time

0     2019-08-01
1     2019-08-02
2     2019-08-05
3     2019-08-06
4     2019-08-07
         ...    
98    2019-12-25
99    2019-12-26
100   2019-12-27
101   2019-12-30
102   2019-12-31
Name: time, Length: 103, dtype: datetime64[ns]

In [30]:
plotter = ReturnPlotter(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
plotter.plot()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]
[1.0, 0.9918788178383535, 0.9782281699092548, 0.979541682850182, 0.9790049786377603, 1.003869919847463, 0.9952473429610538, 1.0394336358179443, 1.0375269234843403, 1.05715899862293, 1.0610571660605206, 1.0713392888669184, 1.0802090321669435, 1.0800254228311146, 1.0765650930405002, 1.1034850464319765, 1.1291267963701848, 1.1074397090498218, 1.1184703929945978, 1.1108788531478409, 1.1195438014194417, 1.1409978461212529, 1.1433918293845555, 1.1420430069559693, 1.133879453409131, 1.1522121394018576, 1.1537375092687407, 1.1507362028176973, 1.1399879947741958, 1.095716959146923, 1.118717559408

ValueError: x and y must have same first dimension, but have shapes (103,) and (104,)

In [ ]:
# ticket: SSE 50：000016
plotter.plot("000016")

### CSI 300

In [ ]:
baseline_df = plotter.get_baseline("399300")

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found


OSError: 获取失败，请检查网络.

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats 
perf_stats_all = perf_func(returns=daily_return, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")

NameError: name 'baseline_df' is not defined

In [ ]:
daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(returns=daily_return_base, factor_returns=daily_return_base, positions=None, transactions=None, turnover_denom="AGB")

print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")

NameError: name 'baseline_df' is not defined